# total number of images in each four class

In [1]:
import os

class_dist = {}
def image_counter(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png']

    for root, dirs, _ in os.walk(folder_path):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            count = 0

            for filename in os.listdir(dir_path):
                file_ext = os.path.splitext(filename)[1].lower()

                if file_ext in image_extensions:
                    count += 1
            
            class_dist[dir_name] = count

    return class_dist

# Set the path to your dataset directory
dataset_dir = "Dataset"

# Call the image_counter function to count images in each class
class_counts = image_counter(dataset_dir)

# Print the count of images in each class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} images")


Class 'Moderate_Demented': 64 images
Class 'Mild_Demented': 896 images
Class 'Very_Mild_Demented': 2240 images
Class 'Non_Demented': 3200 images


# Data augmentation was performed in order to eliminate the class imbalance issue.

In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths
input_dir = 'Dataset'
output_dir = 'Augmentated_data'

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define the number of augmented images per input image
num_augmented_images_per_input = 50

# Define the total number of augmented images to create
total_augmented_images = 3200

# Loop through each class folder
class_names = os.listdir(input_dir)
for class_name in class_names:
    # Set the paths for input and output folders
    input_class_dir = os.path.join(input_dir, class_name)
    output_class_dir = os.path.join(output_dir, class_name)
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_class_dir, exist_ok=True)
    
    # Get a list of image files in the input class folder
    image_files = [f for f in os.listdir(input_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Calculate how many input images are available in this class
    num_input_images = len(image_files)
    
    # Calculate how many augmented images need to be generated for this class
    num_augmented_images_for_class = min(total_augmented_images, num_input_images * num_augmented_images_per_input)
    
    # Loop through each image in the class folder
    for image_file in image_files:
        # Load the input image
        img_path = os.path.join(input_class_dir, image_file)
        img = image.load_img(img_path)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        # Generate 50 augmented images for each input image
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            augmented_image_array = batch[0].astype('uint8')
            output_image_filename = os.path.join(output_class_dir, f'{i}_{image_file}')
            image.save_img(output_image_filename, augmented_image_array)
            
            # Stop after generating the desired number of augmented images for this class
            if i >= num_augmented_images_per_input - 1:
                break

        # Decrement the remaining number of augmented images to create
        num_augmented_images_for_class -= num_augmented_images_per_input
        
        # Stop processing this class if the desired number of augmented images is reached
        if num_augmented_images_for_class <= 0:
            break

    # Stop processing classes if the desired total number of augmented images is reached
    if total_augmented_images <= 0:
        break

print("Augmentation completed.")


Augmentation completed.


In [5]:
import os

class_dist = {}
def image_counter(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png']

    for root, dirs, _ in os.walk(folder_path):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            count = 0

            for filename in os.listdir(dir_path):
                file_ext = os.path.splitext(filename)[1].lower()

                if file_ext in image_extensions:
                    count += 1
            
            class_dist[dir_name] = count

    return class_dist

# Set the path to your dataset directory
dataset_dir = "Augmentated_data"

# Call the image_counter function to count images in each class
class_counts = image_counter(dataset_dir)

# Print the count of images in each class
for class_name, count in class_counts.items():
    print(f"Class '{class_name}': {count} images")


Class 'Mild_Demented': 3200 images
Class 'Moderate_Demented': 3200 images
Class 'Non_Demented': 3200 images
Class 'Very_Mild_Demented': 3200 images


# Feature Extraction using DenseNet - 121 Architecture 

In [7]:
import os
import numpy as np
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input

# Set the paths
input_dir = 'Augmentated_data'
output_dir = 'Extracted_Features'

# Create a DenseNet-121 model (pre-trained on ImageNet)
base_model = DenseNet121(weights='imagenet', include_top=False)

# Define a new model that extracts features from the desired layer
# In this case, we use the GlobalAveragePooling2D layer to obtain feature vectors
feature_layer = GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=feature_layer)

# Loop through each class folder
class_names = os.listdir(input_dir)
for class_name in class_names:
    # Set the paths for input and output folders
    input_class_dir = os.path.join(input_dir, class_name)
    output_class_dir = os.path.join(output_dir, class_name)
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_class_dir, exist_ok=True)
    
    # Get a list of image files in the input class folder
    image_files = [f for f in os.listdir(input_class_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Loop through each image in the class folder
    for image_file in image_files:
        # Load and preprocess the input image
        img_path = os.path.join(input_class_dir, image_file)
        img = image.load_img(img_path, target_size=(224, 224))  # Ensure the target size matches the DenseNet-121 input size
        x = image.img_to_array(img)
        x = preprocess_input(x)
        x = np.expand_dims(x, axis=0)
        
        # Extract features from the input image
        features = model.predict(x)
        
        # Save the extracted feature vector as a numpy file
        output_feature_filename = os.path.splitext(image_file)[0] + '_features.npy'
        output_feature_path = os.path.join(output_class_dir, output_feature_filename)
        np.save(output_feature_path, features)

print("Feature extraction completed.")


1/1 [==============================] - 0s 106ms/step


1/1 [==============================] - 0s 95ms/step


1/1 [==============================] - 0s 99ms/step


1/1 [==============================] - 0s 110ms/step


1/1 [==============================] - 0s 124ms/step


...

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 2s 2s/step
Feature extraction completed.


In [3]:
import numpy as np

# Load the features from the .npy file
features = np.load('Extracted_Features/Mild_Demented/0_mild_103_features.npy')
# Print the shape of the features
print(features.shape)

# Print the first few elements of the features
print(features)

(1, 1024)
[[2.5023063e-04 2.9348489e-03 9.6130493e-04 ... 1.2788525e+00
  5.7265073e-01 3.0032009e-01]]
